# Local run Qwen 0.5b on GRPO

---
이 노트북은 [Original Notebook](https://colab.research.google.com/drive/1bfhs1FMLW3FGa8ydvkOZyBNxLYOu0Hev?usp=sharing#scrollTo=Q7qTZbUcg5VD) 인 gsm8k 수학 데이터셋을 사용하여 Qwen-0.5b 를 학습시키는 내용을 SageMaker 에서 재구성한 내용 입니다.
실제로 [QRPO](https://medium.com/data-science-in-your-pocket/what-is-grpo-the-rl-algorithm-used-to-train-deepseek-12acc19798d3) 를 사용한 학습을 로컬 노트북에서 실행 하는 노트북 입니다.

### 노트북 실험 환경
- SageMaker Studio 의 [Code Editor](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/code-editor.html) 의 ml.g5.12xlarge 를 사용하였습니다.

---


## 1. 환경 설정
- 시작을 위해서 여기 가이드 보세요: [Setup Guide](../setup/README.md)
- 이후에 아래 셀 실행을 통해서, 필요한 패키지가 설치 되었는지를 확인 합니다.

In [1]:
! pip list | grep -E "sagemaker|vllm|trl|datasets"

datasets                           3.2.0
sagemaker                          2.232.2
sagemaker-core                     1.0.21
sagemaker-mlflow                   0.1.0
trl                                0.14.0
vllm                               0.7.2


In [2]:
%load_ext autoreload
%autoreload 2

## 2. train.py 테스트 실행
- train.py 파일이 기본적으로 동작 테스트를 로컬에서 합니다. 문법, 데이터 확인 등의 기본적인 작업을 하고, 이후에 SageMaker Training Job 을 통해서 최종 확인 합니다. 

In [2]:
%%sh
python scripts/train.py num_generations 2

===== CUDA 환경 진단 정보 =====
NVCC 버전 확인 실패: [Errno 2] No such file or directory: 'nvcc'


NVIDIA-SMI 정보:
Mon Mar  3 11:43:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10G                    On  |   00000000:00:1B.0 Off |                    0 |
|  0%   19C    P8             15W /  300W |       1MiB /  23028MiB |      0%      Default |
|                                         |                        |                  N/A |
+--------------------------------

/home/sagemaker-user/.conda/envs/qrpo-sm-train/lib/python3.10/site-packages/trl/trainer/grpo_trainer.py:301: UserWarning: The requested device cuda:0 is also used for training. This may lead to unexpected behavior. It is recommended to use a dedicated device for vLLM.
  warnings.warn(


INFO 03-03 11:43:56 config.py:542] This model supports multiple tasks: {'reward', 'generate', 'classify', 'score', 'embed'}. Defaulting to 'generate'.
INFO 03-03 11:43:56 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='Qwen/Qwen2.5-0.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-0.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda:0, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=Qwen/Qwen2.5-0.5B-Instruct, num_schedule

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  7.21it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  7.20it/s]



...

{'loss': 0.0044, 'grad_norm': 5.25, 'learning_rate': 4.555005188619776e-06, 'completion_length': 123.625, 'rewards/xmlcount_reward_func': 0.16306250169873238, 'rewards/soft_format_reward_func': 0.0, 'rewards/strict_format_reward_func': 0.0, 'rewards/int_reward_func': 0.25, 'rewards/correctness_reward_func': 0.5625, 'reward': 0.9755625054240227, 'reward_std': 0.5905225537717342, 'kl': 0.10875582695007324, 'epoch': 0.27}


 27%|██▋       | 128/467 [16:02<40:15,  7.13s/it]

-------------------- Question:
Ivy baked 20 cupcakes in the morning and fifteen more cupcakes in the afternoon than in the morning. How many cupcakes did she bake? 
Answer:
55 
Response:
<reasoning>
Ivy baked 20 cupcakes in the morning and 20+15=35 cupcakes in the afternoon.
</reasoning>
<answer>
35
</answer> 
Extracted:
35
-------------------- Question:
John gets a new EpiPen every 6 months.  They cost $500 and insurance covers 75%.  How much does he pay per year? 
Answer:
250 
Response:
<reasoning>
Calculating John's annual medical expense involves several steps:

1. **Determine the cost of the EpiPen**: Since it costs $500, this is the amount paid by the patient.

2. **Annual coverage**: Insurance covers 75% of the cost.

3. **Calculate the annual coverage amount**: Multiply the cost by 0.75.

4. **Coverage for each EpiPen**: John gets 1 EpiPen coverage every 6 months.

5. **Calculate the annual allocation**: For each covered EpiPen, John pays a fraction of the coverage amount.

6. 

{'loss': 0.0039, 'grad_norm': 4.75, 'learning_rate': 4.544298464161079e-06, 'completion_length': 127.875, 'rewards/xmlcount_reward_func': 0.18765624798834324, 'rewards/soft_format_reward_func': 0.0, 'rewards/strict_format_reward_func': 0.0, 'rewards/int_reward_func': 0.25, 'rewards/correctness_reward_func': 0.25, 'reward': 0.6876562535762787, 'reward_std': 0.489804033190012, 'kl': 0.09851875714957714, 'epoch': 0.28}


 28%|██▊       | 129/467 [16:09<40:16,  7.15s/it]